In [99]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


##### Define Foursquare Credentials and Version

In [214]:
# @hidden_cell
CLIENT_ID = 'V53MHPBBL23EVUFTF31HTKJAFRJIK2QL1WJURTZ3ARRFF3KE' # Foursquare ID
CLIENT_SECRET = 'G5Z0ALDAAKEOPESPXKBZPXNB2TIEJKGWM33ZGSBCNW0VGW0T' # Foursquare Secret
VERSION = '20180604'
LIMIT = 500

In [165]:
# Pull API keys from the file
with open('4square_keys.txt','r') as f:
    CLIENT_ID, CLIENT_SECRET = [l.strip() for l in f.readlines()]

VERSION = '20180604'
LIMIT = 500

#### Explore venues around my work place 

In [304]:
address = 'Russia, Taganrog'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, ',', longitude)

47.2153657 , 38.9285216


In [49]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            500, 
            LIMIT)

In [33]:
import json, codecs
def json_to_file(data, name='work_nbrs.json'):
    with open(name, 'wb') as f:
        json.dump(data, codecs.getwriter('utf-8')(f), ensure_ascii=False)

In [249]:
def searchVenuesInRect(sw, ne):
    
    venues_list=[]
    
    def get_category(cat_list):
        if len(cat_list) == 0:
            return None
        else:
            return cat_list[0]['name']
              
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&intent=browse&sw={},{}&ne={},{}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            sw[0], 
            sw[1], 
            ne[0], 
            ne[1],
            LIMIT)
            
    # make the GET request
    results = requests.get(url).json()['response']['venues']
        
    # return only relevant information for each nearby venue
    venues_list.append([(
        v['name'], 
        v['location']['lat'], 
        v['location']['lng'],  
        get_category(v['categories']),
        v['id']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
        'Venue', 'Latitude', 'Longitude', 'Category', 'Id']
    
    return(nearby_venues)
        

In [248]:
def searchVenuesInCircle(c, r):
    
    venues_list=[]
    
    def get_category(cat_list):
        if len(cat_list) == 0:
            return None
        else:
            return cat_list[0]['name']
              
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            c[0], 
            c[1], 
            r,             
            LIMIT)
            
    # make the GET request
    results = requests.get(url).json()['response']['venues']
        
    # return only relevant information for each nearby venue
    venues_list.append([(
        v['name'], 
        v['location']['lat'], 
        v['location']['lng'],  
        get_category(v['categories']),
        v['id']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
        'Venue', 'Latitude', 'Longitude', 'Category', 'Id']
    
    return(nearby_venues)
        

In [305]:
sw = [47.2153657 , 38.9285216]
ne = [47.2153657 , 38.9285216]
venues = searchVenuesInRect(sw=sw, ne=ne)
venues.head()

,Venue,Latitude,Longitude,Category,Id
0,Площадь перед администрацией города,47.215733,38.928230,Plaza,5368f4ad498ea0cb80cef632
1,Культ вина,47.215510,38.929310,Wine Bar,5c74142e60255e002c1aefbc
2,Театр имени А. П. Чехова,47.216325,38.928217,Theater,4dcbe98a1f6ea1401d49d12a
3,Администрация Таганрога,47.215517,38.928420,City Hall,4da693d90cb66f658708dafc
4,Л'Этуаль,47.215416,38.929266,Cosmetics Shop,4f83002ee4b0b2237e8a6cb1


#### Create a map of Taganrog city

In [168]:
# plot venues on map
def plot_venues(venues, sw, ne, r=None):
    map_tag = folium.Map(location=sw, zoom_start=13)

    # add bounding box or circle to map
    if r == None:
        folium.Rectangle([sw, ne], color='red').add_to(map_tag)
    else:
        folium.Circle(sw, r, color='red').add_to(map_tag)
    
    # add markers to map
    for lat, lng, name, cat in zip(venues['Venue Latitude'], venues['Venue Longitude'], venues['Venue'], venues['Venue Category']):
        label = '{}, {}'.format(name, cat)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_tag)  
        
    return map_tag

In [211]:
# get bounding box
sw_cur=[ne_tag[0]-0.005, ne_tag[1]-0.005]
ne_cur=ne_tag
#center = [40.7896239, -73.9598939] # NY
#center = [47.2213858, 39.7114196]  #Rostov-on-don

# get venues
venues = getVenuesInRect(sw=sw_cur, ne=ne_cur)
#venues = getVenuesInCircle(c=center, r=500)

# plot venues on map
#plot_venues(venues=venues, sw=center, ne=center, r=500)
plot_venues(venues=venues, sw=sw_cur, ne=ne_cur)

In [208]:
center = [47.2153657, 38.9285216]
venues = getVenuesInRect(sw=center, ne=[center[0]+0.005, center[1]+0.005])
plot_venues(venues=venues, sw=center, ne=[center[0]+0.005, center[1]+0.005])

### Coordinate Grid

In [216]:
url_bounds = 'https://nominatim.openstreetmap.org/search?format=json&q=Taganrog, Russia&polygon_geojson=1'
# get borders in json format
bounds = requests.get(url_bounds).json()[0]['boundingbox']
# convert to float
city_rect = [float(i) for i in bounds] 

In [246]:
print(np.arange(city_rect[0], city_rect[1], 0.005))
print(np.arange(city_rect[2], city_rect[3], 0.005))
print('bounding box =', city_rect)

[47.1887122 47.1937122 47.1987122 47.2037122 47.2087122 47.2137122
 47.2187122 47.2237122 47.2287122 47.2337122 47.2387122 47.2437122
 47.2487122 47.2537122 47.2587122 47.2637122 47.2687122 47.2737122
 47.2787122 47.2837122 47.2887122]
[38.7929772 38.7979772 38.8029772 38.8079772 38.8129772 38.8179772
 38.8229772 38.8279772 38.8329772 38.8379772 38.8429772 38.8479772
 38.8529772 38.8579772 38.8629772 38.8679772 38.8729772 38.8779772
 38.8829772 38.8879772 38.8929772 38.8979772 38.9029772 38.9079772
 38.9129772 38.9179772 38.9229772 38.9279772 38.9329772 38.9379772
 38.9429772 38.9479772 38.9529772 38.9579772 38.9629772 38.9679772]
bounding box = [47.1887122, 47.2899427, 38.7929772, 38.9701473]


In [ ]:
step = 0.005
city_venues = pd.DataFrame(columns=['Venue', 'Latitude', 'Longitude', 'Category', 'Id'])
for lat in np.arange(city_rect[0], city_rect[1], step):
    for lon in np.arange(city_rect[2], city_rect[3], step):
        loc_venues = searchVenuesInRect(sw=[lat, lon], ne=[lat+step, lon+step])
        city_venues = city_venues.append(loc_venues)

print(city_venues.shape)
city_venues.head()

In [ ]:
city_venues = city_venues.drop_duplicates('Id')
city_venues = city_venues.reset_index(drop=True)

In [302]:
city_venues.to_csv('city_venues.csv')

In [306]:
sw

[47.2153657, 38.9285216]

In [307]:
map_tag = folium.Map(location=[47.2153657, 38.9285216], zoom_start=13)

for lat in np.arange(city_rect[0], city_rect[1], step):
    for lon in np.arange(city_rect[2], city_rect[3], step):
        folium.Rectangle([[lat, lon], [lat+step, lon+step]], color='red').add_to(map_tag)
        
map_tag